# Feature Selection

Featuretools provides users with the ability to remove features that are unlikely to be useful in building an effective machine learning model. Reducing the number of features in the feature matrix can both produce better results in the model as well as reduce the computational cost involved in prediction.

Featuretools enables users to perform feature selection on the results of Deep Feature Synthesis with three functions:

- `ft.selection.remove_highly_null_features`
- `ft.selection.remove_single_value_features`
- `ft.selection.remove_highly_correlated_features`

Each of the functions takes in a calculated feature matrix and can optionally take in the list of Feature definitions as well. If just a feature matrix is used as input, then just a feature matrix will be returned, but if both the matrix and the Feature definitions are used, then the results of feature selection will, similarly, be the matrix and the Feature definitions.

We will describe each of these functions in depth, but first we must create an entity set with which we can run `ft.dfs`.

In [1]:
import pandas as pd
import featuretools as ft

from featuretools.selection import (
    remove_low_information_features,
    remove_highly_correlated_features,
    remove_highly_null_features,
    remove_single_value_features,
)

from featuretools.primitives import NaturalLanguage


df1 = pd.DataFrame({'id': [0, 1, 2, 3], 
                    'categories':['a','a','b','b'], 
                    'bools':[True, True, False, False],
                    'half_nulls': [None, None, 88, 100],
                    'quarter_nulls': [None, 1,1, 1],
                    'all_nulls': pd.Series([None, None, None,None], dtype='float'),
                    "diff_ints": [34, 11, 29, 91],
                   
                   })
df2 = pd.DataFrame({
    'id': [0, 1, 2, 3],
    "first_id": [0, 1, 1, 3],
    "words": ["test", "this is a short sentence", "foo bar", "baz"],
    "corr_words": [4, 24, 7, 3],
    'corr_1': [99, 100, 77, 33],
    'corr_2': [99, 100, 77, 33],
})

entities = {
        "first": (df1, 'id'),
        "second": (df2, 'id',  None, {'words': NaturalLanguage}),
    }

es = ft.EntitySet("data", entities, )
es

Entityset: data
  Entities:
    first [Rows: 4, Columns: 7]
    second [Rows: 4, Columns: 6]
  Relationships:
    No relationships

## Remove Highly Null Features

We might have a dataset with columns that have many null values. Deep Feature Synthesis might build features off of those null columns, creating even more highly null features. In this case, we might want to remove any columns whose null values pass a certain threshold. Below is our feature matrix with such a case:

In [2]:
fm, features = ft.dfs(entityset=es,
                          target_entity="first",
                          trans_primitives=['add_numeric'],
                          agg_primitives=[],
                          max_depth=2)
fm

,categories,bools,half_nulls,quarter_nulls,all_nulls,diff_ints,all_nulls + diff_ints,all_nulls + half_nulls,all_nulls + quarter_nulls,diff_ints + half_nulls,diff_ints + quarter_nulls,half_nulls + quarter_nulls
id,,,,,,,,,,,,
0,a,True,NaN,NaN,NaN,34,NaN,NaN,NaN,NaN,NaN,NaN
1,a,True,NaN,1.0,NaN,11,NaN,NaN,NaN,NaN,12.0,NaN
2,b,False,88.0,1.0,NaN,29,NaN,NaN,NaN,117.0,30.0,89.0
3,b,False,100.0,1.0,NaN,91,NaN,NaN,NaN,191.0,92.0,101.0


We look at the above feature matrix and decide to remove the highly null features

In [3]:
ft.selection.remove_highly_null_features(fm)

,categories,bools,half_nulls,quarter_nulls,diff_ints,diff_ints + half_nulls,diff_ints + quarter_nulls,half_nulls + quarter_nulls
id,,,,,,,,
0,a,True,NaN,NaN,34,NaN,NaN,NaN
1,a,True,NaN,1.0,11,NaN,12.0,NaN
2,b,False,88.0,1.0,29,117.0,30.0,89.0
3,b,False,100.0,1.0,91,191.0,92.0,101.0


Notice that calling `remove_highly_null_features` didn't remove every column that contains a null value. By default, we only remove columns where the percentage of null values is above 95%. If we want to lower that threshold, we can set the `pct_null_threshold` paramter ourselves.

In [4]:
remove_highly_null_features(fm, pct_null_threshold=.5)

,categories,bools,quarter_nulls,diff_ints,diff_ints + quarter_nulls
id,,,,,
0,a,True,NaN,34,NaN
1,a,True,1.0,11,12.0
2,b,False,1.0,29,30.0
3,b,False,1.0,91,92.0


Now we're left with a feature matrix containing mostly populated data!

## Remove Single Value Features

Another situation we might run into is one where our calculated features don't have any variance. In those cases, we are likely to want to remove the uninteresting columns. For that, we use `remove_single_value_features`.

In [5]:
fm, features = ft.dfs(entityset=es,
                          target_entity="first",
                          trans_primitives=['is_null'],
                          agg_primitives=[],
                          max_depth=2)
fm

,categories,bools,half_nulls,quarter_nulls,all_nulls,diff_ints,IS_NULL(all_nulls),IS_NULL(bools),IS_NULL(categories),IS_NULL(diff_ints),IS_NULL(half_nulls),IS_NULL(id),IS_NULL(quarter_nulls)
id,,,,,,,,,,,,,
0,a,True,NaN,NaN,NaN,34,True,False,False,False,True,False,True
1,a,True,NaN,1.0,NaN,11,True,False,False,False,True,False,False
2,b,False,88.0,1.0,NaN,29,True,False,False,False,False,False,False
3,b,False,100.0,1.0,NaN,91,True,False,False,False,False,False,False


The example of using `IsNull` as a primitive highlights a case where many columns all have the same value. Lets remove them:

In [6]:
remove_single_value_features(fm)

,categories,bools,half_nulls,diff_ints,IS_NULL(half_nulls),IS_NULL(quarter_nulls)
id,,,,,,
0,a,True,NaN,34,True,True
1,a,True,NaN,11,True,False
2,b,False,88.0,29,False,False
3,b,False,100.0,91,False,False


Notice that we've actually lost two of the three columns with null values because, with the function used as it is above, null values are not considered in whether a column has only one unique value. If we'd like to consider `NaN` its own value, we can set `count_nan_as_value` to `True` and we'll see `quarter_null` back in the matrix.

In [7]:
remove_single_value_features(fm, count_nan_as_value=True)

,categories,bools,half_nulls,quarter_nulls,diff_ints,IS_NULL(half_nulls),IS_NULL(quarter_nulls)
id,,,,,,,
0,a,True,NaN,NaN,34,True,True
1,a,True,NaN,1.0,11,True,False
2,b,False,88.0,1.0,29,False,False
3,b,False,100.0,1.0,91,False,False


## Remove Highly Correlated Features

The last feature selection function we have allows us to remove columns that would likely be redundant to the model we're attempting to build by considering the correlation between pairs of calculated features.

When two columns are determined to be highly correlated, we remove the more complex of the two. For example, say we have two features: `col` and `-(col)`.

We can see that `-(col)` is just the negation of `col`, and so we can guess those columns are going to be highly correlated. `-(col)` has has the `Negate` primitive applied to it, so it is more complex than the identity feature `col`. Therefore, if we only want one of `col` and `-(col)`, we should keep the identity feature. For features that don't have an obvious difference in complexity, we discard the feature that comes later in the feature matrix. 

Let's try this out on our data:

In [8]:
fm, features = ft.dfs(entityset=es,
                          target_entity="second",
                          trans_primitives=['negate', 'num_characters'],
                          agg_primitives=[],
                          max_depth=2)
fm

,first_id,corr_words,corr_1,corr_2,-(corr_1),-(corr_2),-(corr_words),-(first_id),NUM_CHARACTERS(words)
id,,,,,,,,,
0,0,4,99,99,-99,-99,-4,0,4
1,1,24,100,100,-100,-100,-24,-1,24
2,1,7,77,77,-77,-77,-7,-1,7
3,3,3,33,33,-33,-33,-3,-3,3


Note that we have some pretty clear correlations here between all the columns and their negations, but we also have `corr_words` that matches the feature `NUM_CHARACTERS(words)`.

Now, using `remove_highly_correlated_features`, our default threshold for correlation is 95% correlated, and we get all of the obviously correlated features removed, leaving just the less complex features.

In [9]:
remove_highly_correlated_features(fm)

,first_id,corr_words,corr_1
id,,,
0,0,4,99
1,1,24,100
2,1,7,77
3,3,3,33


#### Change the correlation threshold

We can lower the threshold at which to remove correlated features if we'd like to be more restrictive by using the `pct_corr_threshold` parameter.

In [10]:
remove_highly_correlated_features(fm,pct_corr_threshold=.9)

,first_id,corr_words
id,,
0,0,4
1,1,24
2,1,7
3,3,3


#### Check a Subset of Features

If we only want to check a subset of columns, we can set `features_to_check` to the list of columns whose correlation we'd like to check, and no columns outside of that list will be removed.

In [11]:
remove_highly_correlated_features(fm, features_to_check=['corr_1', 'corr_2', '-(corr_1)' ,'-(corr_2)'])

,first_id,corr_words,corr_1,-(corr_words),-(first_id),NUM_CHARACTERS(words)
id,,,,,,
0,0,4,99,-4,0,4
1,1,24,100,-24,-1,24
2,1,7,77,-7,-1,7
3,3,3,33,-3,-3,3


#### Protect Features from Removal

To avoid having specific columns removed from the feature matrix, we can include a list of `features_to_keep`, and these features will not be removed

In [12]:
remove_highly_correlated_features(fm, features_to_keep=['-(corr_1)' ,'-(corr_2)'])

,first_id,corr_words,corr_1,-(corr_1),-(corr_2)
id,,,,,
0,0,4,99,-99,-99
1,1,24,100,-100,-100
2,1,7,77,-77,-77
3,3,3,33,-33,-33
